In [1]:
# Para gestionar ficheros
# -----------------------------------------------------------------------
import os
import glob

# Para trabajar con access porque realmente se tendría que hacer desde la terminal pero esto nos lo simula y por eso es mejor hacerlo por aquí porque lo simplifica
# -----------------------------------------------------------------------
import subprocess


# Para trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd


# Para trabajar con regex
# -----------------------------------------------------------------------
import re

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os


# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd

# Para poner barras progreso
# -----------------------------------------------------------------------
from tqdm import tqdm


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

Acces es como SQL y excel (tiene muchas tablas )

Esto no funciona para windows

In [8]:
resultado= subprocess.run(
    ["mdb-tables",      #esto es lo que nos permite listar las tablas (siempre tienen que ir de la mano con el -1)
    "-1", #para que nos devuelva una lista de las tablas que tenemos
    r"C:\Users\jaime\Desktop\Teoria\Modulo5\teoria\datos\DatosDescomprimidos\2016\SIAE_2016anonimizada.mdb"],    #ruta al fichero
    capture_output=True,    #muestra el mensaje de si hubo un error o si fue todo bien en el proceso de extracción
    text= True)         #devuelve el output en formato de texto          

resultado

FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado

In [9]:
type(resultado)

subprocess.CompletedProcess

Esto nos casa los nombres de las tablas del archivo acces

In [14]:
tablas= resultado.stdout.split("\n")        #porque vemos que las tablas están separadas por \n
tablas

['']

Intentamos sacar todas las tablas de las que hay en el archivo acces

In [13]:
for tabla in tablas:
    with open (f"C:\Users\jaime\Desktop\Teoria\Modulo5\teoria\datos\DatosDescomprimidos\2016\{tabla}.txt", "w") as t: #aquí estmos creando un archivo csv donde se encuentre la tabla
        subprocess.run(["mdb-export",      
        r"C:\Users\jaime\Desktop\Teoria\Modulo5\teoria\datos\DatosDescomprimidos\2016.zip",  # donde quiero guardarlo
        'C1_01_FILIACION anonimizada'],     #nombre de la tabla que vamos a coger para sacarle los datos 
        stdout= t)


FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado

In [16]:
os.getcwd()   #nos dice done está leyendo

'c:\\Users\\jaime\\Desktop\\Teoria\\Modulo5\\teoria'

In [19]:
ruta_origen= r"datos\DatosDescomprimidos"
for i in os.listdir(r"C:\Users\jaime\Desktop\Teoria\Modulo5\teoria\datos\DatosDescomprimidos"):
    ruta_anios= os.path.join(ruta_origen, i)
    if len(os.listdir(ruta_anios)) != 1:
        pass
    else:
        print(ruta_anios)
        print("acces")

datos\DatosDescomprimidos\2014.zip
acces
datos\DatosDescomprimidos\2016.zip
acces
datos\DatosDescomprimidos\2017.zip
acces
datos\DatosDescomprimidos\2018.zip
acces
datos\DatosDescomprimidos\2019.zip
acces
datos\DatosDescomprimidos\2020.zip
acces
datos\DatosDescomprimidos\2021.zip
acces


Hacemos funcion para que vaya por todos los años cogiendo las tablas

In [20]:
def leer_tablas(ruta):
    resultado= subprocess.run(
        ["mdb-tables",      #esto es lo que nos permite listar las tablas (siempre tienen que ir de la mano con el -1)
        "-1", #para que nos devuelva una lista de las tablas que tenemos
        ruta],    #ruta al fichero
        capture_output=True,    #muestra el mensaje de si hubo un error o si fue todo bien en el proceso de extracción
        text= True)         #devuelve el output en formato de texto          
    return resultado.stdout.split("\n")


In [22]:
def exportar_tablas(ruta_guardar, tabla):
    with open (f"{ruta_guardar}\{tabla}.txt", "w") as t: #aquí estmos creando un archivo csv donde se encuentre la tabla
        subprocess.run(["mdb-export",      
        ruta_guardar,  # donde quiero guardarlo
        'C1_01_FILIACION anonimizada'],     #nombre de la tabla que vamos a coger para sacarle los datos 
        stdout= t)

Con esto ya extraemos toda la info de cada tabla de cada archivo access de cada año

In [ ]:
ruta_origen= r"datos\DatosDescomprimidos"
for i in os.listdir(r"C:\Users\jaime\Desktop\Teoria\Modulo5\teoria\datos\DatosDescomprimidos"):
    ruta_anios= os.path.join(ruta_origen, i)
    if len(os.listdir(ruta_anios)) != 1:
        pass
    else:                                    #esto se aplica si solo hay un elemento
        ruta_access= os.path.join(ruta_anios, os.listdir(ruta_anios)[0])
        tablas= leer_tablas(ruta_access)
        for tabla in tablas[:-1]:    #esto es porque hay un txt vacío en lo último
            exportar_tablas(ruta_anios, tabla)

AHORA QUE YA TENEMOS TODOS LOS AÑOS CON TODAS LAS TABLAS CON SU CORRESPONDIENTE INFO. VAMOS A LIMPIAR

Urgencia  
HospDom  
Hospital  
ConsultExt  
CM  
HospDia  
700,701,702  

In [32]:
df_prueba= pd.read_csv("datos/DatosDescomprimidos/2016/C1_20_INGRESOS anonimizada.txt")
df_prueba.columns= [col.lower() for col in df_prueba.columns]
df_700=df_prueba.filter(regex= r"7(00|01|02)")        #con esto filtramos por los que tengan 700, 701 y 702

lista= ["Hospital", "consultExter", "CMA", "hospDia", "Urgencia", "hospDom"]
lista_columnas= df_700.columns
df_700.filter(items= lista)
df_final= pd.DataFrame()

for elemento in lista:

    df_filtro= df_700.filter(like=elemento.lower())

    columnas= df_filtro.columns
    patron = "\d+"
    nuevas_columnas=["_".join(re.findall(patron, columna)[0]) for columna in columnas]
    df_filtro.columns= nuevas_columnas
    df_filtro["tipo"]= elemento        #Para crear una columna con el 
    df_filtro[["año", "ncodi"]]= df_prueba[["año", "ncodi"]]
    

    df_final= pd.concat([df_final, df_filtro])   #hay que poner el axis=1 porque sino, los indices no están bien y nos genera nulos

TypeError: concat() got an unexpected keyword argument 'index'

In [30]:
df_final.head(2)

,7_0_0,7_0_1,7_0_1,7_0_1,7_0_2,tipo,año,ncodi,tipo,año,ncodi,7_0_0,7_0_1,7_0_1,7_0_1,7_0_2,tipo,año,ncodi,7_0_0,7_0_1,7_0_1,7_0_1,7_0_2,tipo,año,ncodi,7_0_0,7_0_1,7_0_1,7_0_1,7_0_2,tipo,año,ncodi,7_0_0,7_0_1,7_0_1,7_0_1,7_0_2,tipo,año,ncodi
0,0.0,13230,1633.0,11597.0,4285.0,Hospital,2016,989,consultExter,2016,989,0.0,204,204.0,0,559,CMA,2016,989,0.0,0,0,0.0,0,hospDia,2016,989,3658,85770,4592,81178,40983,Urgencia,2016,989,0.0,0,0.0,0,0,hospDom,2016,989
1,17342.0,134496,71296.0,63200.0,35089.0,Hospital,2016,990,consultExter,2016,990,0.0,0,0.0,0,0,CMA,2016,990,0.0,3900,3900,0.0,0,hospDia,2016,990,4722,93813,22166,71647,59009,Urgencia,2016,990,0.0,0,0.0,0,0,hospDom,2016,990


In [ ]:
for año in os.listdir(r"datos\DatosDescomprimidos"):
    ruta_años= os.path.join(r"datos\DatosDescomprimidos", año)
    for file in os.listdir(ruta_años):
        if 'ingresos' in file.lower():
           ruta_archivo= os.path
                      